In [1]:
import eventstudy as es
import numpy as np
import matplotlib.pyplot as plt
import datetime
import yfinance as yf
import pandas as pd

检测输入数据

In [2]:
BSE = yf.download('MME=F', '2022-01-01', '2022-08-22')['Close']
ITC = yf.download('^BSESN', '2022-01-01', '2022-08-22')['Close']
print(type(BSE))
BSE

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
<class 'pandas.core.series.Series'>


Date
2022-01-03    1235.199951
2022-01-04    1230.400024
2022-01-05    1210.800049
2022-01-06    1216.599976
2022-01-07    1228.000000
                 ...     
2022-08-15    1019.000000
2022-08-16    1019.599976
2022-08-17    1015.099976
2022-08-18    1008.400024
2022-08-19     994.900024
Name: Close, Length: 159, dtype: float64

In [3]:
# to calcualte the return fo teh ITC and BSE sensex
ITC = ITC.pct_change()
BSE = BSE.pct_change()
ITC= pd.DataFrame(ITC)
BSE = pd.DataFrame(BSE)
BSE =BSE.dropna(axis=0)
ITC =ITC.dropna(axis=0)

# To have equal counts in ITC & BSE
df = ITC['Close'],BSE['Close']
df1 = pd.DataFrame(df)
df2 = df1.T
df2.columns = ['ITC','BSE']
df3 = df2.dropna()
df3

,ITC,BSE
Date,,
2022-01-04,0.011367,-0.003886
2022-01-05,0.006135,-0.015930
2022-01-06,-0.010317,0.004790
2022-01-07,0.002396,0.009370
2022-01-10,0.010896,-0.000081
...,...,...
2022-08-12,0.002194,0.010653
2022-08-16,0.006381,0.000589
2022-08-17,0.006984,-0.004413


模拟输入数据

In [4]:
###todo：数据要求
###todo：交易日期为 index，每一列的 column name 为股票代码，每一列是 close【收盘价】的percentage changed【百分比变化，相对于前一天】
stockData=pd.read_csv("stockData/TushareRawData.csv")
stockData=stockData[['ts_code','trade_date','pct_chg']].rename(columns={'trade_date':'Date'})
# companyList=[]
afterColumnChanged=pd.DataFrame()
# print("total number of day: {}".format(len(stockData['Date'].values)))
# for stockCode in list(stockData.groupby('ts_code')):
#     theData={'Date':stockCode[1]['Date'].values,stockCode[0]:stockCode[1]['pct_chg'].values}
#     dataByCompany=pd.DataFrame(theData)
#     dataByCompany['Date']=pd.to_datetime(dataByCompany["Date"], format="%Y-%m-%d")
#     dataByCompany=dataByCompany.dropna(axis=1)
#     # dataByCompany=dataByCompany.set_index('Date')
#     # afterColumnChanged=afterColumnChanged.append(dataByCompany)
#     companyList.append(dataByCompany)
#     # afterColumnChanged=pd.concat([afterColumnChanged,dataByCompany])
#     # print("{} is finished".format(stockCode[0]))
#
# afterColumnChanged=pd.concat(companyList[:100])
# # afterColumnChanged=afterColumnChanged.drop_duplicates()
# afterColumnChanged=afterColumnChanged.sort_values(by='Date',ascending=False)
# # afterColumnChanged=afterColumnChanged.dropna(how='all')
stockData['Date']=pd.to_datetime(stockData["Date"], format="%Y-%m-%d")
stockData=stockData.set_index('Date')
# dt = datetime.datetime(2022, 12, 30)
# # for ele in afterColumnChanged[afterColumnChanged['Date']==dt].values:
# #     print(ele)
# afterColumnChanged

for stockCode in stockData.groupby('ts_code'):
    # if len(stockCode[1]['pct_chg'].values) >= 1000:
    afterColumnChanged[stockCode[0]]=stockCode[1]['pct_chg']

# afterColumnChanged=afterColumnChanged.dropna()
afterColumnChanged

/var/folders/ct/s3lrfn2541b8hydxbp_njzch0000gn/T/ipykernel_27166/779191946.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  afterColumnChanged[stockCode[0]]=stockCode[1]['pct_chg']
/var/folders/ct/s3lrfn2541b8hydxbp_njzch0000gn/T/ipykernel_27166/779191946.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  afterColumnChanged[stockCode[0]]=stockCode[1]['pct_chg']
/var/folders/ct/s3lrfn2541b8hydxbp_njzch0000gn/T/ipykernel_27166/779191946.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of

,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,...,872392.BJ,872808.BJ,872925.BJ,873001.BJ,873122.BJ,873169.BJ,873223.BJ,873305.BJ,873339.BJ,873527.BJ
Date,,,,,,,,,,,,,,,,,,,,,
2022-12-30,0.9977,0.2203,1.2552,0.5405,2.7732,0.0000,0.4367,0.9182,3.5912,1.0462,...,-2.5,2.3968,2.1127,0.5618,0.0000,0.0000,0.6270,0.6965,1.2121,-0.3713
2022-12-29,-0.8371,-2.2605,0.1047,-0.5376,-9.9853,-0.7653,-0.8658,-0.5809,-3.7234,-4.8922,...,NaN,0.4073,0.0000,-1.6575,-2.7539,-0.5017,-2.4465,-0.9852,-1.0000,0.2481
2022-12-28,0.2288,0.3782,-1.5464,-1.5873,2.5602,0.0000,-1.2821,-1.7129,-2.5907,1.0897,...,NaN,-0.1848,-0.7687,-7.2983,-0.0860,0.5042,-2.3881,-1.4563,-0.7937,0.1242
2022-12-27,2.6625,0.7621,2.5370,2.7174,9.9338,-0.3812,0.8621,-2.8526,0.5208,2.1404,...,NaN,-0.4781,-0.3482,-8.3333,-0.1717,0.0000,0.6006,2.7944,-0.1980,-0.1241
2022-12-26,-1.6179,-2.1832,1.2848,0.0000,2.1997,-0.6313,0.8696,1.2029,3.7838,-0.8489,...,NaN,-0.4029,0.6307,NaN,4.0000,0.6768,0.0000,-0.5952,-0.3945,0.1242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-01-10,0.0600,NaN,-0.4800,0.3400,1.0000,6.5300,0.6400,0.0000,NaN,0.2800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-09,-0.8800,NaN,0.0000,0.0000,-1.3800,-0.3900,1.4700,1.7000,NaN,-1.1200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-08,-1.8400,NaN,1.1000,0.3400,3.2500,8.0100,1.5900,-0.4500,NaN,0.7100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# afterColumnChanged.to_csv("eventStudyData/train/incomeImport.csv",sep=',',index =True)

In [6]:
afterColumnChanged=pd.read_csv("eventStudyData/train/incomeImport.csv",index_col="Date")
print(afterColumnChanged.index.dtype)
afterColumnChanged

object


,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,...,872392.BJ,872808.BJ,872925.BJ,873001.BJ,873122.BJ,873169.BJ,873223.BJ,873305.BJ,873339.BJ,873527.BJ
Date,,,,,,,,,,,,,,,,,,,,,
2022-12-30,0.9977,0.2203,1.2552,0.5405,2.7732,0.0000,0.4367,0.9182,3.5912,1.0462,...,-2.5,2.3968,2.1127,0.5618,0.0000,0.0000,0.6270,0.6965,1.2121,-0.3713
2022-12-29,-0.8371,-2.2605,0.1047,-0.5376,-9.9853,-0.7653,-0.8658,-0.5809,-3.7234,-4.8922,...,NaN,0.4073,0.0000,-1.6575,-2.7539,-0.5017,-2.4465,-0.9852,-1.0000,0.2481
2022-12-28,0.2288,0.3782,-1.5464,-1.5873,2.5602,0.0000,-1.2821,-1.7129,-2.5907,1.0897,...,NaN,-0.1848,-0.7687,-7.2983,-0.0860,0.5042,-2.3881,-1.4563,-0.7937,0.1242
2022-12-27,2.6625,0.7621,2.5370,2.7174,9.9338,-0.3812,0.8621,-2.8526,0.5208,2.1404,...,NaN,-0.4781,-0.3482,-8.3333,-0.1717,0.0000,0.6006,2.7944,-0.1980,-0.1241
2022-12-26,-1.6179,-2.1832,1.2848,0.0000,2.1997,-0.6313,0.8696,1.2029,3.7838,-0.8489,...,NaN,-0.4029,0.6307,NaN,4.0000,0.6768,0.0000,-0.5952,-0.3945,0.1242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-01-10,0.0600,NaN,-0.4800,0.3400,1.0000,6.5300,0.6400,0.0000,NaN,0.2800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-09,-0.8800,NaN,0.0000,0.0000,-1.3800,-0.3900,1.4700,1.7000,NaN,-1.1200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-08,-1.8400,NaN,1.1000,0.3400,3.2500,8.0100,1.5900,-0.4500,NaN,0.7100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


模拟三因子数据 an aggregate of event

In [10]:
eventAggregate=pd.DataFrame()
companyTickers=afterColumnChanged.columns
companyTickers

Index(['000001.SZ', '000002.SZ', '000004.SZ', '000005.SZ', '000006.SZ',
       '000007.SZ', '000008.SZ', '000009.SZ', '000010.SZ', '000011.SZ',
       ...
       '872392.BJ', '872808.BJ', '872925.BJ', '873001.BJ', '873122.BJ',
       '873169.BJ', '873223.BJ', '873305.BJ', '873339.BJ', '873527.BJ'],
      dtype='object', length=5061)

输入数据

In [7]:
# es.Single.import_returns('returns.csv')
# es.Single.import_FamaFrench('famafrench.csv')

In [8]:
# event = es.Single.FamaFrench_3factor(
#     security_ticker = 'AAPL',
#     event_date = np.datetime64('2013-03-04'),
#     event_window = (-2,+10),
#     estimation_size = 300,
#     buffer_size = 30
# )

In [9]:
# event.plot(AR=True)
# plt.show()